### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']



In [2]:
# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()



In [3]:
# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [5]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1]# number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features)# user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [6]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 41.945635
2 		 14.550152
3 		 4.043056
4 		 2.664310
5 		 2.486258
6 		 2.407188
7 		 2.337770
8 		 2.264605
9 		 2.184239
10 		 2.095557


In [7]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.9650681    8.7764198   10.66982786  10.20458177]
 [  8.85610788   6.65685969   9.16768772   9.05424872]
 [  7.6012131    7.18190225   8.4380754    7.60141625]
 [  9.66740068   8.1539164    9.737312     9.84691863]
 [  8.88625686   6.9863844    8.88469858   8.65596465]
 [  6.38941531   5.62845605   7.95984121   6.70122298]
 [  8.93518802   7.53337216  10.20887545   9.18838205]
 [  8.39061573   6.8722599    8.5947795    8.27217052]
 [  8.50088225   7.50271855   8.86291444   8.49103261]
 [  7.77037954   6.46317531   8.08375031   7.74584561]
 [  8.97252375   7.6016451    9.46082072   8.78934601]
 [  9.28410856   8.87455222  10.07497915   9.48620888]
 [  9.07509297   8.45299451   9.75490609   9.15903789]
 [  6.38550471   6.32209315   7.02895173   6.40617363]
 [ 10.02308931   7.92014231  10.06160137  10.08146681]
 [  9.76904291   8.53419848   9.95361399   9.97085623]
 [  8.77375121   7.38012566   8.26352722   8.32947118]
 [  7.86176706   6.60277048   6.37657089   7.3245461 ]
 [  7.8752

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [8]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 45.631208
2 		 15.542008
3 		 4.147909
4 		 2.810658
5 		 2.673580
6 		 2.626665
7 		 2.591080
8 		 2.554195
9 		 2.512866
10 		 2.465950
11 		 2.412736
12 		 2.352679
13 		 2.285407
14 		 2.210795
15 		 2.129031
16 		 2.040683
17 		 1.946729
18 		 1.848537
19 		 1.747798
20 		 1.646397
21 		 1.546241
22 		 1.449082
23 		 1.356359
24 		 1.269103
25 		 1.187909
26 		 1.112980
27 		 1.044222
28 		 0.981351
29 		 0.924002
30 		 0.871797
31 		 0.824396
32 		 0.781505
33 		 0.742873
34 		 0.708272
35 		 0.677483
36 		 0.650276
37 		 0.626402
38 		 0.605597
39 		 0.587578
40 		 0.572056
41 		 0.558743
42 		 0.547357
43 		 0.537635
44 		 0.529330
45 		 0.522222
46 		 0.516113
47 		 0.510827
48 		 0.506213
49 		 0.502139
50 		 0.498491
51 		 0.495170
52 		 0.492089
53 		 0.489174
54 		 0.486358
55 		 0.483581
56 		 0.480788
57 		 0.477929
58 		 0.474956
59 		 0.471823
60 		 0.468486
61 		 0.464901
62 		 0.461023
63 		 0.456810
64 		

In [9]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00000298  10.00000167  10.00000105   9.99999653]
 [  9.99995173   3.99997684   8.99998594  10.00005229]
 [  7.99999478   8.99999825   9.99999908   5.00000522]
 [  9.0000036    8.00000287  10.00000208   9.99999583]
 [ 10.00003597   5.00001837   9.00001148   8.99996087]
 [  5.99999167   3.99999605   9.99999764   6.00000908]
 [  9.00002717   8.00001331  10.00000814   8.99997043]
 [ 10.00002868   5.0000135    9.0000081    7.99996899]
 [  6.99995243   7.99997666   9.9999856    8.0000514 ]
 [  9.00000952   5.00000443   9.00000267   6.99998972]
 [  9.00001194   8.00000576  10.00000345   7.9999869 ]
 [  9.00002915  10.00001395  10.00000835   8.99996816]
 [  9.9999897    8.99999431   9.99999617   8.00001095]
 [  5.00001975   8.00000927   5.00000551   7.99997846]
 [  9.99999772   7.99999812   9.9999985   10.00000236]
 [  8.99998661   8.99999296   9.99999544  10.00001435]
 [  8.99998583   7.99999291   7.99999553   8.00001515]
 [  9.99999007   7.99999507   0.99999686  10.00001045]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [10]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [11]:
# run SVD on the matrix with the missing value
user_mat, movie_mat =  FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 42.089585
2 		 14.421411
3 		 4.155855
4 		 2.943510
5 		 2.842861
6 		 2.822535
7 		 2.812128
8 		 2.803508
9 		 2.795048
10 		 2.786235
11 		 2.776740
12 		 2.766243
13 		 2.754397
14 		 2.740818
15 		 2.725083
16 		 2.706715
17 		 2.685183
18 		 2.659894
19 		 2.630197
20 		 2.595382
21 		 2.554699
22 		 2.507379
23 		 2.452675
24 		 2.389919
25 		 2.318595
26 		 2.238428
27 		 2.149489
28 		 2.052283
29 		 1.947827
30 		 1.837665
31 		 1.723825
32 		 1.608685
33 		 1.494766
34 		 1.384483
35 		 1.279901
36 		 1.182548
37 		 1.093326
38 		 1.012523
39 		 0.939916
40 		 0.874918
41 		 0.816732
42 		 0.764481
43 		 0.717306
44 		 0.674412
45 		 0.635099
46 		 0.598764
47 		 0.564892
48 		 0.533051
49 		 0.502880
50 		 0.474081
51 		 0.446415
52 		 0.419696
53 		 0.393788
54 		 0.368601
55 		 0.344086
56 		 0.320230
57 		 0.297052
58 		 0.274593
59 		 0.252912
60 		 0.232077
61 		 0.212159
62 		 0.193228
63 		 0.175343
64 		

In [12]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.639880725921328:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [13]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.325699
2 		 10.827070
3 		 7.464360
4 		 5.767153
5 		 4.691816
6 		 3.939280
7 		 3.380230
8 		 2.948143
9 		 2.604702
10 		 2.325977
11 		 2.096119
12 		 1.904158
13 		 1.742229
14 		 1.604517
15 		 1.486603
16 		 1.385039
17 		 1.297073
18 		 1.220470
19 		 1.153392
20 		 1.094325


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [14]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [15]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
